In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

In [36]:
data_frame = pd.read_csv('..\\data\\000001.csv', encoding='gb18030')
split = int(len(data_frame) * 0.8)
data_frame = data_frame.sort_index(ascending=False)  # 把数据的顺序改成从古至今
cols = ['收盘价', '最高价', '最低价']
data = data_frame.get(cols)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)
train = data[: split]
test = data[split:]

In [3]:
print(train.shape, test.shape)

(5768, 3) (1443, 3)


In [4]:
def create_dataset(data, time_steps, pre_steps, interval=False):
    x, y = [], []
    remainder = (len(data) - time_steps) % pre_steps if interval else 0
    step = pre_steps if interval else 1

    for i in range(0, len(data) - time_steps - pre_steps - remainder + 1, step):
        x.append(data[i: i + time_steps])
        y.append(data[i + time_steps: i + time_steps + pre_steps])

    return np.array(x), np.array(y)

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense

In [7]:
from tensorflow.keras.callbacks import Callback
import warnings


class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [16]:
early_stopping = EarlyStoppingByLossVal(
    monitor='val_loss',
    value=0.001,
    verbose=1,
)
callbacks = [early_stopping]

In [24]:
x_train, y_train = create_dataset(train, 120, 1)
x_test, y_test = create_dataset(test, 120, 1)

In [25]:
print(x_train.shape, y_train.shape)

(5648, 120, 3) (5648, 1, 3)


In [19]:
inputs = Input(shape=(120, 3))
a = LSTM(50, return_sequences=True)(inputs)
a = LSTM(50, return_sequences=True)(a)
a = LSTM(50)(a)
outputs = Dense(3)(a)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='mse', optimizer='adam')

In [20]:
model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=64,
    verbose=1,
    callbacks=callbacks,
    validation_data=(x_test, y_test)
)

Epoch 1/100
89/89 [==============================] - 55s 623ms/step - loss: 0.1185 - val_loss: 0.2116
Epoch 2/100
89/89 [==============================] - 48s 542ms/step - loss: 0.1068 - val_loss: 0.2167
Epoch 3/100
89/89 [==============================] - 48s 541ms/step - loss: 0.1074 - val_loss: 0.2334
Epoch 4/100
89/89 [==============================] - 48s 545ms/step - loss: 0.1063 - val_loss: 0.2548
Epoch 5/100
89/89 [==============================] - 49s 546ms/step - loss: 0.1063 - val_loss: 0.2372
Epoch 6/100
89/89 [==============================] - 48s 544ms/step - loss: 0.1066 - val_loss: 0.2569
Epoch 7/100
89/89 [==============================] - 48s 543ms/step - loss: 0.1064 - val_loss: 0.2420
Epoch 8/100
89/89 [==============================] - 48s 542ms/step - loss: 0.1063 - val_loss: 0.2242
Epoch 9/100
89/89 [==============================] - 48s 544ms/step - loss: 0.1066 - val_loss: 0.2600
Epoch 10/100
89/89 [==============================] - 48s 542ms/step - loss: 0.106

KeyboardInterrupt: 

In [37]:
x_train, y_train = create_dataset(train, 1, 1)
x_test, y_test = create_dataset(test, 1, 1)

In [38]:
print(x_train.shape, y_train.shape)

(5767, 1, 3) (5767, 1, 3)


In [39]:
print(x_train)

[[[0.         0.         0.        ]]

 [[0.00079163 0.00078742 0.00063124]]

 [[0.00169682 0.0016878  0.00154021]]

 ...

 [[0.32833606 0.32659153 0.32987495]]

 [[0.3278215  0.32744457 0.3301936 ]]

 [[0.32586999 0.32535059 0.32760419]]]


In [40]:
inputs2 = Input(shape=(1, 3))
a = LSTM(120)(inputs2)
outputs2 = Dense(3)(a)
model2 = Model(inputs=inputs2, outputs=outputs2)
model2.compile(loss='mse', optimizer='adam')

In [41]:
model2.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=64,
    verbose=1,
    callbacks=callbacks,
    validation_data=(x_test, y_test)
)

Epoch 1/100
91/91 [==============================] - 4s 39ms/step - loss: 0.0428 - val_loss: 0.0395
Epoch 2/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0283 - val_loss: 0.0498
Epoch 3/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0619
Epoch 4/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0627
Epoch 5/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0275 - val_loss: 0.0553
Epoch 6/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0275 - val_loss: 0.0589
Epoch 7/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0275 - val_loss: 0.0540
Epoch 8/100
91/91 [==============================] - 1s 8ms/step - loss: 0.0274 - val_loss: 0.0614
Epoch 9/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0627
Epoch 10/100
91/91 [==============================] - 1s 8ms/step - loss: 0.0275 - val_loss: 0.0629
Epoch 11

91/91 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0605
Epoch 84/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0647
Epoch 85/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0571
Epoch 86/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0646
Epoch 87/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0592
Epoch 88/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0275 - val_loss: 0.0522
Epoch 89/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0275 - val_loss: 0.0629
Epoch 90/100
91/91 [==============================] - 1s 7ms/step - loss: 0.0275 - val_loss: 0.0588
Epoch 91/100
91/91 [==============================] - 1s 8ms/step - loss: 0.0275 - val_loss: 0.0568
Epoch 92/100
91/91 [==============================] - 1s 8ms/step - loss: 0.0274 - val_loss: 0.0565
Epoch 93/100


In [ ]:
inputs3 = Input(shape=(1, 3))
a = LSTM(120)(inputs)
a = RepeatVector(10)(a)
a = LSTM(360, return_sequences=True)(a)
outputs3 = TimeDistributed(Dense(3))(a)
model3 = Model(inputs=inputs3, outputs=outputs3)
model3.compile(loss='mse', optimizer='adam')

In [ ]:
model3.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=64,
    verbose=1,
    callbacks=callbacks,
    validation_data=(x_test, y_test)
)